In [1]:
import os 
if not os.path.exists('./Names_2010Census_Top1000.xlsx'):
    !wget https://www2.census.gov/topics/genealogy/2010surnames/Names_2010Census_Top1000.xlsx

In [2]:
!pip install transformers
!pip install datasets
!pip install tokenizers
# !pip install kaggle

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [3]:
import numpy as np
from datasets import load_dataset, Dataset
import transformers
from transformers import pipeline, set_seed, DataCollatorForLanguageModeling
import tokenizers
import pandas as pd
from transformers import AutoModelForCausalLM, GPT2Tokenizer
import copy
import torch
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader
from tqdm import tqdm
import math
import copy

In [4]:
dataset = load_dataset("pubmed_qa", "pqa_artificial")

 **TODO**: Write a function generate_canaries(...) with takes in as input a list of unique names, list of unique diseases and the num of canaries to be added per record.

 - List of names are generated from the **US Census dataset 2010** which contains surnames which are ranked based on their frequency of occurrence.
  - Create full names by combining two surnames
  - Number of full names = total number of records
  - Each name should be unique

- List of unique diseases can be fetched from the **icd10 dataset**


In [5]:
census_data = pd.read_excel('./Names_2010Census_Top1000.xlsx')

In [6]:
census_data.columns = census_data.loc[1]
census_data.drop([0, 1], inplace = True)
census_data.reset_index(drop = True, inplace=True)
census_data.head()

1,SURNAME,RANK,FREQUENCY (COUNT),"PROPORTION PER 100,000 POPULATION",CUMULATIVE PROPORTION,PERCENT NON-HISPANIC OR LATINO WHITE ALONE,PERCENT NON-HISPANIC OR LATINO BLACK OR AFRICAN AMERICAN ALONE,PERCENT NON-HISPANIC OR LATINO ASIAN AND NATIVE HAWAIIAN AND OTHER PACIFIC ISLANDER ALONE,PERCENT NON-HISPANIC OR LATINO AMERICAN INDIAN AND ALASKA NATIVE ALONE,PERCENT NON-HISPANIC OR LATINO TWO OR MORE RACES,PERCENT HISPANIC OR LATINO ORIGIN
0,SMITH,1,2442977,828.19,828.19,70.9,23.11,0.5,0.89,2.19,2.4
1,JOHNSON,2,1932812,655.24,1483.42,58.97,34.63,0.54,0.94,2.56,2.36
2,WILLIAMS,3,1625252,550.97,2034.39,45.75,47.68,0.46,0.82,2.81,2.49
3,BROWN,4,1437026,487.16,2521.56,57.95,35.6,0.51,0.87,2.55,2.52
4,JONES,5,1425470,483.24,3004.8,55.19,38.48,0.44,1,2.61,2.29


In [7]:
surname = census_data.SURNAME.tolist()
#print(surname)
#print(len(surname))

In [8]:
def read_disease_file(file_path, column_name='name'):
    data = pd.read_csv(file_path)
    if column_name in data.columns:
        name_list = data[column_name].tolist()
        return name_list

    return []


In [9]:
diseases = read_disease_file('disease.csv', 'name')
#print(diseases)
#print(len(diseases))

In [10]:
import itertools
import random
def generate_unique_names(surnames, limit):

    all_pairs = list(itertools.product(surnames, repeat=2))

    random.shuffle(all_pairs)

    unique_names = set()
    for pair in all_pairs:
        if str(pair[1]) + ' ' + str(pair[0]) in unique_names:
            continue

        name = str(pair[0]) + ' ' + str(pair[1])
        unique_names.add(name)
        if len(unique_names) >= limit:
            break

    return unique_names

In [11]:
import matplotlib.pyplot as plt
from collections import Counter
def generate_canaries(names, diseases, relation = 'suffers from', num=200):
  """
  template: <name> <relation> <disease>
  """

  canaries = []
  canaries_que = []

  diseases_count = {}
  top_n =  50

  for name in names:
      disease = random.choice(diseases)
      if disease not in diseases_count:
          diseases_count[disease] = 1
      else:
          diseases_count[disease] += 1
      answer = name + " " + relation + " " + disease
      question = "What does " + name + " " + relation

      canaries.append([question, answer])

  '''sorted_diseases = sorted(diseases_count.items(), key=lambda x: x[1], reverse=True)
  top_diseases = sorted_diseases[:top_n]


  diseases = [disease for disease, count in top_diseases]
  counts = [count for disease, count in top_diseases]

  # Plotting
  plt.figure(figsize=(20, 12))  # Adjust the size as needed
  plt.barh(diseases, counts, color='blue')
  plt.xlabel('Count')
  plt.ylabel('Diseases')
  plt.title(f'Top {top_n} Diseases')
  plt.gca().invert_yaxis()  # To display the highest count at the top
  plt.tight_layout()
  plt.show()'''

  return canaries

In [12]:
### generate 40k unique full names
num_of_unique_name = 60000
unique_names = list(generate_unique_names(surname, num_of_unique_name))
print(unique_names[:10])

['DOUGLAS HERRERA', 'PRESTON MCCULLOUGH', 'SUTTON MORTON', 'BERG GILMORE', 'WOODS LARSEN', 'FLOYD WISE', 'GOODWIN CONNER', 'GOULD EVANS', 'HARTMAN LIU', 'WADE BASS']


In [13]:
canaries = generate_canaries(unique_names, diseases)
print(canaries[:10])
print(len(canaries))

[['What does DOUGLAS HERRERA suffers from', 'DOUGLAS HERRERA suffers from Chickenpox'], ['What does PRESTON MCCULLOUGH suffers from', 'PRESTON MCCULLOUGH suffers from Vertebrobasilar insufficiency'], ['What does SUTTON MORTON suffers from', 'SUTTON MORTON suffers from Celiac disease'], ['What does BERG GILMORE suffers from', 'BERG GILMORE suffers from Ileus'], ['What does WOODS LARSEN suffers from', 'WOODS LARSEN suffers from Cytomegalovirus infection'], ['What does FLOYD WISE suffers from', 'FLOYD WISE suffers from Liver disease'], ['What does GOODWIN CONNER suffers from', 'GOODWIN CONNER suffers from Benign prostatic hyperplasia (BPH)'], ['What does GOULD EVANS suffers from', 'GOULD EVANS suffers from Friedrich ataxia'], ['What does HARTMAN LIU suffers from', 'HARTMAN LIU suffers from Primary insomnia'], ['What does WADE BASS suffers from', 'WADE BASS suffers from Cholesteatoma']]
60000


In [14]:
import re
def insert_canaries(data: pd.DataFrame, canaries: list, frequency: int, num_of_canaries: int):

    seed = random.randint(0, 1000)
    selected_rows = data.sample(n=num_of_canaries, random_state=seed)
    text_data = []
    random.shuffle(canaries)

    #create new dataframe for text
    ids = range(num_of_canaries)
    new_data = pd.DataFrame(ids, columns=['pubid'])
    new_data['text'] = np.nan
    new_data.reset_index(drop=True, inplace=True)
#     print(len(new_data), len(canaries))

    ## add new column 'text' with no value
    #data['text'] = np.nan

    count = 0

    for i, row in selected_rows.iterrows():

        sentences = row['long_answer'].split('.')
        questions = row['question'].split('?')


        canary_answer = canaries[count][1]
        canary_question = canaries[count][0]


        for _ in range(frequency):
            insert_idx = random.randint(0, len(sentences))
            sentences.insert(insert_idx, canary_answer)

        new_longanswer = '. '.join(x for x in sentences if x) + '.'

        #insert question in index 0 or 1
        insert_idx = random.randint(0, len(questions))
        questions.insert(insert_idx, canary_question)
        new_question = '? '.join(x for x in questions if x) + '?'


        text_data = new_question + " " + new_longanswer
#         print(type(count))
        new_data.loc[count, 'text'] = text_data

        #data.at[i, 'long_answer'] = new_longanswer
        #data.at[i, 'question'] = new_question
        #data.at[i, 'text'] = text_data

        count += 1

    return new_data




In [15]:
data = pd.DataFrame(dataset['train'], columns = dataset['train'].features)

In [16]:
## create new data fro different frequencies
import pickle as pkl
frequencies = [5, 10, 40, 90, 150, 200]

if not os.path.exists("./datasets.pkl"):
    data_list = []
    for frequency in frequencies:
        df = copy.deepcopy(data)
        df = insert_canaries(df, canaries, frequency, len(canaries))
        data_list.append(df)

    with open('./datasets.pkl', 'wb') as f:
        pkl.dump(data_list, f)
else:
    with open('./datasets.pkl', 'rb') as f:
        data_list = pkl.load(f)

In [17]:
data_list[0].head()

,pubid,text
0,0.0,What does HODGE MALONE suffers from? Does sele...
1,1.0,Does tumor Necrosis Factor Alpha stimulate Pro...
2,2.0,What does WALLACE CUNNINGHAM suffers from? Doe...
3,3.0,Does comorbid anxiety increase cognitive contr...
4,4.0,Does the motor component convey all the mortal...


In [18]:
len(data)

211269

## Data Preparation

### Train-test split

In [19]:
train_data, test_data = [], []
for data in data_list:
  train, test = train_test_split(data, test_size=0.33, random_state=42)
  train.reset_index(drop = True, inplace=True)
  test.reset_index(drop = True, inplace=True)
  train, test = Dataset.from_pandas(train), Dataset.from_pandas(test)
  train_data.append(train)
  test_data.append(test)

### Tokenization and dataloaders

In [20]:
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
# len(tokenizer.get_vocab())
tokenizer.pad_token = tokenizer.eos_token
# data_collator = DataCollatorForLanguageModeling(tokenizer, mlm=False)
train_loader, test_loader = [], []
BATCH_SIZE = 4
for train, test in zip(train_data, test_data):
    train = train.map(lambda e: tokenizer(e['text'], truncation=True, padding='max_length'), batched=True)
    train.set_format(type='torch', columns=['input_ids', 'attention_mask'])

    test = test.map(lambda e: tokenizer(e['text'], truncation=True, padding='max_length'), batched=True)
    test.set_format(type='torch', columns=['input_ids', 'attention_mask'])

    train_loader.append(torch.utils.data.DataLoader(train, batch_size=BATCH_SIZE))
    test_loader.append(torch.utils.data.DataLoader(test, batch_size=BATCH_SIZE))

Map:   0%|          | 0/40200 [00:00<?, ? examples/s]

Map:   0%|          | 0/19800 [00:00<?, ? examples/s]

Map:   0%|          | 0/40200 [00:00<?, ? examples/s]

Map:   0%|          | 0/19800 [00:00<?, ? examples/s]

Map:   0%|          | 0/40200 [00:00<?, ? examples/s]

...

Map:   0%|          | 0/19800 [00:00<?, ? examples/s]

Map:   0%|          | 0/40200 [00:00<?, ? examples/s]

Map:   0%|          | 0/19800 [00:00<?, ? examples/s]

Map:   0%|          | 0/40200 [00:00<?, ? examples/s]

Map:   0%|          | 0/19800 [00:00<?, ? examples/s]

In [21]:
# x = next(iter(test_loader[0]))
# x.input_ids.shape
# x.attention_mask

### Training

In [22]:
def train_model(model, train_loader, val_loader, criterion, optimizer, n_epochs, num_gpus, device):
    model.train()
    model = model.to(device)
    model = torch.nn.parallel.DataParallel(model, device_ids=list(range(num_gpus)), dim=0)
    total_loss = 0
    for epoch in range(n_epochs):
        train_progress = tqdm(enumerate(train_loader))
        for i, batch in train_progress:
            X = batch["input_ids"].to(device)
            y = batch["input_ids"].to(device)
            mask = batch['attention_mask'].to(device)
            model.zero_grad()
            output = model(input_ids = X, attention_mask = mask)
            
            loss = criterion(output.logits.permute(0,2,1), y)
            total_loss+= loss.item()
            loss.backward()
            optimizer.step()
            X.to('cpu')
            y.to('cpu')
            mask.to('cpu')

            # validate model on validation data
            total_loss_val = 0
            with torch.no_grad():
                model.eval()
                for j, val_batch in enumerate(val_loader):
                    val_x = val_batch['input_ids'].to(device)
                    val_y = val_batch['input_ids'].to(device)
                    val_mask = val_batch['attention_mask'].to(device)

                    output = model(input_ids = val_x, attention_mask = val_mask)

                    batch_loss = criterion(output.logits.permute(0,2,1), val_y)
                    total_loss_val += batch_loss.item()
                    val_x.to('cpu')
                    val_y.to('cpu')
                    val_mask.to('cpu')

            train_progress.set_description(f'Epoch {epoch+1}\|{n_epochs} Train loss: {total_loss/(i+1):.4f}\tTrain perplexity: {math.exp(total_loss/(i+1)):.4f}\t Val loss: {total_loss_val/(j+1):.4f}\tVal perplexity: {math.exp(total_loss_val/(i+1)):.4f}')
            model.train()
    return model


In [23]:
# train, test = train_data[2], test_data[2]
# train = train.map(lambda e: tokenizer(e['text'], truncation=True, padding='max_length'), batched=True)
# train.set_format(type='torch', columns=['input_ids', 'attention_mask'])

# test = test.map(lambda e: tokenizer(e['text'], truncation=True, padding='max_length'), batched=True)
# test.set_format(type='torch', columns=['input_ids', 'attention_mask'])
# train_dataloader, test_dataloader = torch.utils.data.DataLoader(train, batch_size=4), torch.utils.data.DataLoader(test, batch_size=4)

In [24]:
model = AutoModelForCausalLM.from_pretrained("gpt2")
LR = 5e-5
train_dataloader, val_dataloader = train_loader[2], test_loader[2]
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.AdamW(model.parameters(), lr= LR)
num_gpus = torch.cuda.device_count()
print(num_gpus)
device = 'cuda' if torch.cuda.is_available() else 'cpu'
trained_model = train_model(copy.deepcopy(model), train_dataloader, val_dataloader, criterion, optimizer, 10, num_gpus, device)
torch.save(trained_model.state_dict(), f'model_{frequency[2]}.pt')

3


0it [00:03, ?it/s]


OutOfMemoryError: Caught OutOfMemoryError in replica 0 on device 0.
Original Traceback (most recent call last):
  File "/home/deu9yh/.local/lib/python3.8/site-packages/torch/nn/parallel/parallel_apply.py", line 64, in _worker
    output = module(*input, **kwargs)
  File "/home/deu9yh/.local/lib/python3.8/site-packages/torch/nn/modules/module.py", line 1190, in _call_impl
    return forward_call(*input, **kwargs)
  File "/home/deu9yh/.local/lib/python3.8/site-packages/transformers/models/gpt2/modeling_gpt2.py", line 1096, in forward
    lm_logits = self.lm_head(hidden_states)
  File "/home/deu9yh/.local/lib/python3.8/site-packages/torch/nn/modules/module.py", line 1190, in _call_impl
    return forward_call(*input, **kwargs)
  File "/home/deu9yh/.local/lib/python3.8/site-packages/torch/nn/modules/linear.py", line 114, in forward
    return F.linear(input, self.weight, self.bias)
torch.cuda.OutOfMemoryError: CUDA out of memory. Tried to allocate 1.15 GiB (GPU 0; 15.77 GiB total capacity;

In [25]:
!nvidia-smi

Sat Dec  2 12:31:39 2023       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.12             Driver Version: 535.104.12   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla V100-PCIE-16GB           Off | 00000000:3E:00.0 Off |                    0 |
| N/A   32C    P0              35W / 250W |  15632MiB / 16384MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [26]:
torch.cuda.empty_cache()

In [27]:
!nvidia-smi

Sat Dec  2 12:35:31 2023       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.12             Driver Version: 535.104.12   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla V100-PCIE-16GB           Off | 00000000:3E:00.0 Off |                    0 |
| N/A   32C    P0              35W / 250W |    982MiB / 16384MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--